<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Ragas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.ragas.io/en/v0.1.21/concepts/metrics/answer_relevance.html

In [ ]:
pip install langchain_openai ragas

In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    FactualCorrectness,
    AnswerCorrectness
)

# OpenAI API Anahtarını Ayarla
os.environ["OPENAI_API_KEY"] = "sk"

# Excel dosyasını oku (B sütununda sorular, C sütununda referans cevaplar, D sütununda deepseek-r1 cevapları)
df = pd.read_excel("/content/Veri Karşılaştırma_test.xlsx")  # Dosya yolunuza göre güncelleyiniz

# Kolonları uygun değişkenlere alalım
questions = df.iloc[:, 1].tolist()
reference_answers = df.iloc[:, 2].tolist()
deepseek_answers = df.iloc[:, 3].tolist()

# LLM Modelini Başlat
llm = ChatOpenAI(model="gpt-4o")  # GPT-4o değerlendirme için kullanılacak
evaluator_llm = LangchainLLMWrapper(llm)

# Değerlendirme için veri setini oluştur
dataset = []
for question, deepseek_response, reference in zip(questions, deepseek_answers, reference_answers):
    dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": [],  # Bağlam kullanmadığımız için boş liste
            "response": deepseek_response,
            "reference": reference
        }
    )

# Dataset'i Ragas ile formatla
evaluation_dataset = EvaluationDataset.from_list(dataset)

# Ragas değerlendirmesini gerçekleştir
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        AnswerRelevancy(),
        Faithfulness(),
        FactualCorrectness(),
        AnswerCorrectness()
    ],
    llm=evaluator_llm
)

# Sonuçları yazdır
print("\n--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---\n")
print(result)


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]


--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---

{'answer_relevancy': 0.8458, 'faithfulness': 0.0200, 'factual_correctness(mode=f1)': 0.2492, 'answer_correctness': 0.4218}


In [ ]:
# İlk 3 elemanı yazdırma
print("Questions - İlk 3 Satır:", questions[:3])
print("Reference Answers - İlk 3 Satır:", reference_answers[:3])
print("DeepSeek Answers - İlk 3 Satır:", deepseek_answers[:3])

# Son 3 elemanı yazdırma
print("\nQuestions - Son 3 Satır:", questions[-3:])
print("Reference Answers - Son 3 Satır:", reference_answers[-3:])
print("DeepSeek Answers - Son 3 Satır:", deepseek_answers[-3:])


Questions - İlk 3 Satır: ['Sistem mühendisliği nedir ve temel amacı nedir?', 'V-Model (V-Şeması) nedir ve hangi aşamaları içerir?', '“Sistem gereksinimleri” ile “kullanıcı gereksinimleri” arasındaki fark nedir?']
Reference Answers - İlk 3 Satır: ['Sistem mühendisliği, karmaşık sistemlerin tasarım, geliştirme, entegrasyon ve yaşam döngüsü yönetimiyle ilgilenen disiplinler arası bir alanıdır. Temel amacı, sistemleri en verimli şekilde tasarlamak, sürekliliğini sağlamak ve uçtan uca yönetmektir.', 'V-Model, sistem veya yazılım geliştirme yaşam döngüsünü tanımlamak için kullanılan bir modeldir. Sol kolunda gereksinim analizi ve tasarım faaliyetleri, sağ kolunda ise test ve doğrulama faaliyetleri bulunur. Ortada entegrasyon aşaması yer alır ve süreç, gereksinimlerden doğrulamaya kadar sistematik bir biçimde ilerler.', 'Sistem gereksinimleri, bir sistemin teknik ve işlevsel niteliklerini tanımlar ve tasarım ekibi tarafından anlaşılması gereken ayrıntıları içerir. Kullanıcı gereksinimleri ise

In [ ]:
pip install "trulens-eval[openai,langchain]" --upgrade

In [ ]:
# Eski kırıntıları temizle (opsiyonel ama tavsiye edilir)
!pip uninstall -y trulens trulens-eval trulens-providers-openai trulens-apps-langchain

In [ ]:
# Gerekli her şeyi tek seferde yükle
!pip install "trulens-eval==1.4.9" trulens-providers-openai trulens-apps-langchain langchain-community

In [ ]:
#   1) Köprü (back-compat) yolu
from trulens_eval.feedback.provider.openai import OpenAI as TruOpenAI
from trulens_eval import TruBasicApp, Feedback, Select, Tru

#   2) Doğrudan yeni modüler yol
from trulens.providers.openai import OpenAI as TruOpenAI
from trulens.apps.langchain import TruChain          # gerekiyorsa
#from trulens import Tru, Feedback, Select
import pandas as pd

In [ ]:
# --- 1. Veri setini oku -----------------------------------------------------
df = pd.read_excel("/content/Data karşılaştırma.xlsx")

questions         = df.iloc[:, 1].tolist()   # B sütunu
reference_answers = df.iloc[:, 2].tolist()   # C sütunu
model_answers     = df.iloc[:, 3].tolist()   # D sütunu

# --- 2. Hakem LLM ve provider ----------------------------------------------
judge_llm = ChatOpenAI(model="gpt-4o", temperature=0)
provider  = TruOpenAI(llm=judge_llm)

# --- 3. Feedback tanımları --------------------------------------------------
f_relevance = (
    Feedback(
        AnswerRelevance(provider=provider).measure,
        name="Answer Relevance"
    )
    .on_input_output()
)

f_grounded = (
    Feedback(
        Groundedness(provider=provider).measure,
        name="Faithfulness/Groundedness"
    )
    .on_input_output()
)

# Referans cevaba yakınlık: referansı her kaydın inputs'undan çekeceğiz
f_gt_agree = (
    Feedback(
        GroundTruthAgreement(provider=provider).agreement_measure,
        name="Ground-Truth Agreement"
    )
    .on_output()                                 # model cevabı
    .against(lambda rec: rec.inputs["reference"])  # altın cevap
)

# --- 4. Basit Tru app -------------------------------------------------------
tru_app = TruBasicApp(
    app_id   = "deepseek_eval",
    feedbacks= [f_relevance, f_grounded, f_gt_agree]
)

# --- 5. Kayıt döngüsü -------------------------------------------------------
tru = Tru()
tru.reset_database()

for q, ref, ans in zip(questions, reference_answers, model_answers):
    tru_app.record(
        inputs  = {"prompt": q, "reference": ref},
        outputs = {"response": ans}
    )

# --- 6. Sonuçları çek -------------------------------------------------------
records = tru.get_records(app_ids=["deepseek_eval"])

for rec in records:
    print("Soru :", rec.record.inputs["prompt"][:80])
    print("Cevap:", rec.record.outputs["response"][:80])
    for fb in rec.feedback:
        print(f"  {fb.name}: {fb.value:.2f}")
    print("-"*60)


NameError: name 'ChatOpenAI' is not defined

## Sample by Sample Examination

In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    FactualCorrectness,
    AnswerCorrectness
)

# === 1. OpenAI anahtarı ===
os.environ["OPENAI_API_KEY"] = "sk--"  # kendi anahtarını yaz

# === 2. Excel dosyasını oku ===
df = pd.read_excel("/content/Veri Karşılaştırma_test.xlsx", sheet_name="Chatgpt")

# === 3. Değerlendirici modeller ===
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# === 4. Model sütunları ===
model_columns = {
    "deepseek-r1": "Deepseek-r1 (System chatbot)",
    "llama-3.1": "Llama 3.1 8b (System Chatbot)",
    "claude-3.7": "Claude Cevap"
}

# === 5. Her modelin skorlarını hesapla ===
all_results = []

for model_name, col_name in model_columns.items():
    print(f"\n--- {model_name} modeli değerlendiriliyor ---")

    dataset_list = []
    for i, row in df.iterrows():
        dataset_list.append({
            "user_input": str(row["Soru"]),
            "retrieved_contexts": [],  # RAG bağlamı yok
            "response": str(row[col_name]),
            "reference": str(row["Cevap-Chatgpt-o1"])
        })

    evaluation_dataset = EvaluationDataset.from_list(dataset_list)

    result = evaluate(
        dataset=evaluation_dataset,
        metrics=[
            AnswerRelevancy(),
            Faithfulness(),
            FactualCorrectness(),
            AnswerCorrectness()
        ],
        llm=evaluator_llm,
        embeddings=embeddings
    )

    # örnek bazlı skorları yakala
    if hasattr(result, "scores"):
        df_scores = pd.DataFrame(result.scores)
    elif hasattr(result, "to_pandas"):
        df_scores = result.to_pandas()
    elif isinstance(result, dict) and "results" in result:
        df_scores = pd.DataFrame(result["results"])
    else:
        raise ValueError("Skor verisi bulunamadı")

    df_scores["model"] = model_name
    df_scores["dataset"] = "chatgpt"
    df_scores["sample_id"] = df["Soru No"]

    all_results.append(df_scores)

# === 6. Tek dosya olarak kaydet ===
final_df = pd.concat(all_results, ignore_index=True)
final_df.rename(columns={
    "answer_relevancy": "AnswerRelevancy",
    "faithfulness": "Faithfulness",
    "factual_correctness": "FactualCorrectness",
    "answer_correctness": "AnswerCorrectness"
}, inplace=True)

output_path = "/content/ragas_per_sample_chatgpt.xlsx"
final_df.to_excel(output_path, index=False)
print(f"\n✅ Kaydedildi: {output_path}")

from google.colab import files
files.download(output_path)
